In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [3]:
data = pd.read_csv("testdata.manual.2009.06.14.csv", header=None, encoding='latin-1', engine='python', error_bad_lines=False)
data.columns = ['sentimiento', 'id', 'fecha', 'query', 'usuario', 'texto']  # Asignar nombres a las columnas
data = data[['sentimiento', 'texto']]

<ipython-input-3-180ef250418c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("testdata.manual.2009.06.14.csv", header=None, encoding='latin-1', engine='python', error_bad_lines=False)


In [28]:
data['sentimiento'] = data['sentimiento'].replace({0: 'negativo', 2: 'neutral', 4: 'positivo'})

X = data['texto'].values
y = data['sentimiento'].values

In [29]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
"""
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
"""
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y, num_classes=3)

In [31]:
# Pad secuencias para que todas tengan la misma longitud
X = pad_sequences(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
# Construir el modelo
model = Sequential()
model.add(Embedding(5000, 32, input_length=X.shape[1]))
model.add(LSTM(100, return_sequences=True))  # Capa LSTM adicional
model.add(Dropout(0.2))  # Capa de dropout para regularización
model.add(LSTM(50, return_sequences=True))  # Otra capa LSTM
model.add(Dropout(0.2))  # Capa de dropout adicional
model.add(LSTM(25))  # Otra capa LSTM
model.add(Dropout(0.2))  # Capa de dropout adicional
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Pérdida: {loss:.2f}")
print(f"Precisión: {accuracy*100:.2f}%")

Epoch 1/50
25/25 [==============================] - 9s 110ms/step - loss: 1.0958 - accuracy: 0.3266 - val_loss: 1.1061 - val_accuracy: 0.4000
Epoch 2/50
25/25 [==============================] - 1s 58ms/step - loss: 1.0926 - accuracy: 0.3744 - val_loss: 1.1035 - val_accuracy: 0.2600
Epoch 3/50
25/25 [==============================] - 1s 58ms/step - loss: 1.0407 - accuracy: 0.4347 - val_loss: 1.0843 - val_accuracy: 0.3900
Epoch 4/50
25/25 [==============================] - 1s 59ms/step - loss: 0.8072 - accuracy: 0.6583 - val_loss: 1.0210 - val_accuracy: 0.5800
Epoch 5/50
25/25 [==============================] - 2s 91ms/step - loss: 0.4804 - accuracy: 0.8442 - val_loss: 1.2359 - val_accuracy: 0.5000
Epoch 6/50
25/25 [==============================] - 2s 68ms/step - loss: 0.2330 - accuracy: 0.9246 - val_loss: 1.3359 - val_accuracy: 0.5600
Epoch 7/50
25/25 [==============================] - 1s 58ms/step - loss: 0.1217 - accuracy: 0.9698 - val_loss: 1.6378 - val_accuracy: 0.5200
Epoch 8/50
2